In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from IPython.display import IFrame
import ipywidgets as widgets

In [32]:
import json
import networkx as nx
import osmnx as ox
import folium
from folium.features import DivIcon

In [21]:
from geometric import GeometricPoint, GeometricEdge
from surfacemovement import Flight, Trajectory
from mapmatching import WeightBasedMapMatching
from airportgraph import AirportGraph, CHANGI_RUNWAYS_3, DEPARTURE_RUNWAY_MAP

## Load Airport Graph

In [22]:
G = nx.read_yaml('openstreetmap_changi_v2.yaml')
graph = AirportGraph(G)

## Load Flights data

In [42]:
with open('test.json', 'r') as f:
    data = json.load(f)

In [43]:
data['flights'].keys()

dict_keys(['0', '1', '2', '3', '4'])

In [44]:
flight_data = data['flights']['0']

In [45]:
flight = Flight.load_from_dict(flight_data)

## Map-matching object

In [23]:
map_matching = WeightBasedMapMatching(graph)
map_matching.set_verbal(False)

In [48]:

points, points_velocity, timestamp = flight.get_sampling(distance_diff=10)

best_route, accept_point_id = map_matching.run_v2(points, points_velocity)
segments = list(zip(best_route, best_route[1:]))

lat, lng = tuple(points[0].get_coordinate())
graph_map = folium.Map(location=[lat,lng], 
                       zoom_start=18, max_zoom=20, tiles='cartodbpositron')
graph_map = graph.plot_folium(folium_map=graph_map, anotation=False, fit_bounds=False)

# graph_map = graph.plot_folium()
# graph_map = flight.plot_folium(folium_map=graph_map)

# for idx, point in enumerate(points):
for idx, p in enumerate(flight):
    lat, lng = tuple(p.get_coordinate())
    folium.CircleMarker(location=[lat, lng],
                        color='green',
                        fill=True,
                        fill_color='#3186cc',
                        radius=2,
                        weight=1).add_to(graph_map)
    
    folium.map.Marker(
                    [lat, lng],
                    icon=DivIcon(
                        icon_size=(150, 36),
                        icon_anchor=(0, 0),
                        html='<div style="font-size: 8pt">%s</div>' % idx,
                    )
                ).add_to(graph_map)

for edge in segments:
    graph_map = ox.plot_route_folium(G, edge, route_map=graph_map)

filepath = 'offline_map_matching.html'
graph_map.save(filepath)


False


In [49]:
IFrame(filepath, width=800, height=700)